In [1]:
import pandas as pd
import numpy as np
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import joblib

c:\Users\Quirama\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\experimental\enable_hist_gradient_boosting.py:18: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


In [2]:
# 1. Cargar datos
train = pd.read_csv("./data/train.csv")
test = pd.read_csv("./data/test.csv")


In [3]:
# 2. Eliminar columnas con >50% nulos
threshold = 0.5
cols_to_drop = train.columns[train.isnull().mean() > threshold].tolist()
test_cols_to_drop = test.columns[test.isnull().mean() > threshold].tolist()
cols_to_drop = list(set(cols_to_drop + test_cols_to_drop))
train.drop(columns=cols_to_drop, inplace=True)
test.drop(columns=cols_to_drop, inplace=True)

In [4]:
# 3. Guardar target y eliminar ID
X = train.drop(columns=["ID", "RENDIMIENTO_GLOBAL"])
y = train["RENDIMIENTO_GLOBAL"]
X_test = test.drop(columns=["ID"], errors='ignore')

In [5]:
# 4. Codificar variables categóricas con OrdinalEncoder
cat_cols = X.select_dtypes(include=["object", "category"]).columns
encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
X[cat_cols] = encoder.fit_transform(X[cat_cols])
X_test[cat_cols] = encoder.transform(X_test[cat_cols])

In [8]:
# 5. Imputar nulos con KNNImputer
imputer = KNNImputer(n_neighbors=2)
X = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)
X_test = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns)

KeyboardInterrupt: 

In [ ]:
# 6. Entrenar modelo
model = HistGradientBoostingClassifier(random_state=42)
model.fit(X, y)

In [ ]:
# 7. Guardar modelo y datos procesados
joblib.dump(model, "modelo_gb.pkl")
X.to_csv("./data/train_gb_preprocesado.csv", index=False)
X_test.to_csv("./data/test_gb_preprocesado.csv", index=False)
pd.DataFrame(y, columns=["target"]).to_csv("./data/target_gb.csv", index=False)

print("✅ Modelo entrenado y archivos guardados correctamente.")

In [ ]:
# 8. Realizar predicción en el conjunto de prueba
predicciones = model.predict(X_test)

In [ ]:
# 9. Crear el archivo de envío (submission)
submission3=pd.DataFrame({
    "ID": test["ID"],  # Asegúrate de usar la columna ID original del conjunto de prueba
    "target": predicciones
})

In [ ]:
# 10. Guardar el archivo de envío en la carpeta 'data'
submission3.to_csv("./data/submission3.csv", index=False)

# Confirmación del guardado del archivo de envío
print("✅ Archivo 'submission.csv' generado correctamente.")